In [3]:
from keras.models import load_model
from collections import deque 
import numpy as np 
import pickle 
import cv2
import time

In [4]:
model = load_model("./videoClassificationModel")
lb = pickle.load(open("./videoClassificationAndBinarizer.pickle", "rb"))
outputvideo = ("./demoOutput.avi")
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
Queue = deque (maxlen=128)

In [5]:
capture_video=cv2.VideoCapture("./example_clips/DemoVideo.mp4")

In [12]:
import cv2
import numpy as np
import time

outputvideo = "./demoOutput.mp4"
ptime = 0
writer = None
(Width, Height) = (None, None) 
while True:
    # Read a frame from the capture device
    (taken, frame) = capture_video.read()
    if not taken:
        break

    # Get frame dimensions if not already obtained
    if Width is None or Height is None:
        (Width, Height) = frame.shape[:2]

    # Process the frame
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (244, 224)).astype("float32")
    frame -= mean
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Queue.append(preds)
    results = np.array(Queue).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]

    # Apply Gaussian blur only if the label isn't "swimming"
    if label != "swimming":
        output = cv2.GaussianBlur(output, (99, 99), 0)
    size = (Width, Height)
    result = cv2.VideoWriter('filename.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, size)
    # Initialize the video writer if needed

    # Write the processed frame to the video
    result.write(output)

    # Calculate and display FPS
    ctime = time.time()
    fps = int(1 / (ctime - ptime))
    cv2.putText(output, str(fps), (20, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 255), 2)
    ptime = ctime

    # Show the frame and handle user input
    cv2.imshow("In progress", output)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# Release resources when finished
print("Finalizing")
writer.release()  # Release VideoWriter object
capture_video.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 74ms/step
Finalizing


AttributeError: 'NoneType' object has no attribute 'release'

: 